In [1]:
# from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate

chain = (
    PromptTemplate.from_template(
        """Given the user question below, classify it as either being about `LangChain`, `OpenAI`, or `Other`.

Do not respond with more than one word.

<question>
{question}
</question>

Classification:"""
    )
    | ChatOpenAI(model_name="gpt-3.5-turbo")
    | StrOutputParser()
)

chain.invoke({"question": "how do I call openai?"})

'OpenAI'

In [9]:
langchain_chain = PromptTemplate.from_template(
    """You are an expert in langchain. \
Always answer questions starting with "As Harrison Chase told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-3.5-turbo")

openai_chain = PromptTemplate.from_template(
    """You are an expert in openai. \
Always answer questions starting with "As Dario Amodei told me". \
Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-3.5-turbo")

general_chain = PromptTemplate.from_template(
    """Respond to the following question:

Question: {question}
Answer:"""
) | ChatOpenAI(model_name="gpt-3.5-turbo")

In [10]:
def route(info):
    if "openai" in info["topic"].lower():
        return openai_chain
    elif "langchain" in info["topic"].lower():
        return langchain_chain
    else:
        return general_chain

In [11]:
from langchain_core.runnables import RunnableLambda

full_chain = {"topic": chain, "question": lambda x: x["question"]} | RunnableLambda(route)
print(full_chain)

first={
  topic: PromptTemplate(input_variables=['question'], template='Given the user question below, classify it as either being about `LangChain`, `OpenAI`, or `Other`.\n\nDo not respond with more than one word.\n\n<question>\n{question}\n</question>\n\nClassification:')
         | ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000019B1918A3B0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000019B1918BFD0>, openai_api_key=SecretStr('**********'), openai_proxy='')
         | StrOutputParser(),
  question: RunnableLambda(lambda x: x['question'])
} last=RunnableLambda(route)


In [12]:
full_chain.invoke({"question": "how do I use openai?"})

AIMessage(content="As Dario Amodei told me, to use OpenAI, you can start by accessing their API and documentation on their website. This will provide you with the necessary tools and resources to integrate OpenAI's technology into your projects and applications.", response_metadata={'token_usage': {'completion_tokens': 49, 'prompt_tokens': 47, 'total_tokens': 96}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1913a721-4ed6-4940-b24c-4c626c377681-0')

In [13]:
full_chain.invoke({"question": "how do I use LangChain?"})

AIMessage(content='As Harrison Chase told me, to use LangChain, you need to first create an account on the platform and familiarize yourself with the various features and services it offers. From there, you can start connecting with other users, accessing language learning resources, and participating in language exchange opportunities to improve your language skills.', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 44, 'total_tokens': 106}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-2abdf0d3-4551-4547-ba83-4e058832383e-0')

In [14]:
full_chain.invoke({"question": "whats 2 + 2"})

AIMessage(content='4', response_metadata={'token_usage': {'completion_tokens': 1, 'prompt_tokens': 24, 'total_tokens': 25}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-9a04dd0d-2963-47e2-9a18-536fe6f36963-0')